In [105]:
import pandas as pd
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

In [96]:
df=pd.read_csv("original_data.csv")
# df.rename(columns={"Time":"time","Open":"open","High":"high","Low":"low","Last":"close","Volume":"volume","Change":"change"}, inplace=True)
df.rename(columns={"Last":"Close", "Change":"open_close_change_$"}, inplace=True)
len(df)

147047

In [97]:
# Viewing non-trading days
non_trading=df.loc[df["Volume"]==0]
non_trading

,Time,Open,High,Low,Close,open_close_change_$,Volume
87668,12/24/19 15:30,321.23,321.23,321.23,321.23,0.00,0
87669,12/24/19 13:10,321.23,321.23,321.23,321.23,-0.01,0
94121,11/29/19 15:30,314.31,314.31,314.31,314.31,0.00,0
94122,11/29/19 13:10,314.31,314.31,314.31,314.31,-0.02,0
134495,7/3/19 15:30,298.80,298.80,298.80,298.80,0.00,0
134496,7/3/19 13:10,298.80,298.80,298.80,298.80,0.00,0


In [98]:
# Dropping non-trading days from dataset
df.drop(index=non_trading.index, inplace=True)
# Reversing DataFrame
df=df.iloc[::-1]
# Resetting index
df.reset_index(drop=True, inplace=True)
df

,Time,Open,High,Low,Close,open_close_change_$,Volume
0,5/17/19 11:51,287.135,287.41,287.1220,287.3800,0.2401,134011
1,5/17/19 11:52,287.375,287.44,287.2700,287.4300,0.0500,63474
2,5/17/19 11:53,287.430,287.57,287.4050,287.5100,0.0800,116036
3,5/17/19 11:54,287.500,287.58,287.4300,287.5800,0.0700,60199
4,5/17/19 11:55,287.585,287.62,287.4800,287.5000,-0.0800,88436
...,...,...,...,...,...,...,...
147036,11/13/20 15:55,357.920,357.96,357.8100,357.8789,-0.0411,492569
147037,11/13/20 15:56,357.860,357.95,357.8000,357.8200,-0.0589,309294
147038,11/13/20 15:57,357.825,358.21,357.8118,358.2050,0.3850,479111
147039,11/13/20 15:58,358.205,358.35,358.1400,358.3500,0.1450,718921


In [99]:
old_close=1
old_volume=1
close_list=[]
for index, row in df.iterrows():
    close=row["Close"]
    volume=row["Volume"]
    
    # Calculating differences in price and volume
    close_diff=close-old_close
    volume_diff=volume-old_volume
    
    # Creating binary outcome of closing price change compared to previous day
    if close_diff>0:
    # Closed higher
        close_change=1
    else:
    # Closed lower or no change
        close_change=0
        
    # Calculating percent changes
    price_swing=row["High"]-row["Low"]
    price_percent_change=((close-old_close)/old_close)*100
    volume_change_percent=((volume-old_volume)/old_volume)*100
    
    # Calculating moving averages
    if len(close_list)==200:
        close_list.pop(0)
    close_list.append(close)
    # 200 min
    if len(close_list)==200:
        avg_200_day=sum(close_list)/200
    else:
        avg_200_day=sum(close_list[:200])/len(close_list[:200])
    # 50 min
    if len(close_list)>=50:
        avg_50_day=sum(close_list[:50])/50
    else:
        avg_50_day=sum(close_list[:50])/len(close_list[:50])
    # 20 min
    if len(close_list)>=20:
        avg_20_day=sum(close_list[:20])/20
    else:
        avg_20_day=sum(close_list[:20])/len(close_list[:20])
    # 13 min
    if len(close_list)>=13:
        avg_13_day=sum(close_list[:13])/13
    else:
        avg_13_day=sum(close_list[:13])/len(close_list[:13])
    # 8 min
    if len(close_list)>=8:
        avg_8_day=sum(close_list[:8])/8
    else:
        avg_8_day=sum(close_list[:8])/len(close_list[:8])
    # 5 min
    if len(close_list)>=5:
        avg_5_day=sum(close_list[:5])/5
    else:
        avg_5_day=sum(close_list[:5])/len(close_list[:5])
    
    # Adding values to DataFrame
    df.loc[index, "close_change_$"]=close_diff
    df.loc[index, "close_change_%"]=price_percent_change
    df.loc[index, "volume_change"]=volume_diff
    df.loc[index, "volume_change_%"]=volume_change_percent
    df.loc[index, "5_min_avg_$"]=avg_5_day
    df.loc[index, "8_min_avg_$"]=avg_8_day
    df.loc[index, "13_min_avg_$"]=avg_13_day
    df.loc[index, "20_min_avg_$"]=avg_20_day
    df.loc[index, "50_min_avg_$"]=avg_50_day
    df.loc[index, "200_min_avg_$"]=avg_200_day
    df.loc[index, "minute_volatility_$"]=price_swing
    df.loc[index, "price_change_binary"]=close_change
    
    # Replacing closing information
    old_close=close
    old_volume=volume

In [100]:
# The price_change and volume_change columns in the first row are not scaled properly
# This is because there was no previous day's values to compare to
df

,Time,Open,High,Low,Close,open_close_change_$,Volume,close_change_$,close_change_%,volume_change,volume_change_%,5_min_avg_$,8_min_avg_$,13_min_avg_$,20_min_avg_$,50_min_avg_$,200_min_avg_$,minute_volatility_$,price_change_binary
0,5/17/19 11:51,287.135,287.41,287.1220,287.3800,0.2401,134011,286.3800,28638.000000,134010.0,1.340100e+07,287.38000,287.380000,287.380000,287.380000,287.380000,287.380000,0.2880,1.0
1,5/17/19 11:52,287.375,287.44,287.2700,287.4300,0.0500,63474,0.0500,0.017399,-70537.0,-5.263523e+01,287.40500,287.405000,287.405000,287.405000,287.405000,287.405000,0.1700,1.0
2,5/17/19 11:53,287.430,287.57,287.4050,287.5100,0.0800,116036,0.0800,0.027833,52562.0,8.280871e+01,287.44000,287.440000,287.440000,287.440000,287.440000,287.440000,0.1650,1.0
3,5/17/19 11:54,287.500,287.58,287.4300,287.5800,0.0700,60199,0.0700,0.024347,-55837.0,-4.812041e+01,287.47500,287.475000,287.475000,287.475000,287.475000,287.475000,0.1500,1.0
4,5/17/19 11:55,287.585,287.62,287.4800,287.5000,-0.0800,88436,-0.0800,-0.027818,28237.0,4.690609e+01,287.48000,287.480000,287.480000,287.480000,287.480000,287.480000,0.1400,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
147036,11/13/20 15:55,357.920,357.96,357.8100,357.8789,-0.0411,492569,-0.0411,-0.011483,102932.0,2.641741e+01,356.41100,356.460438,356.471977,356.516195,356.648530,357.163053,0.1500,0.0
147037,11/13/20 15:56,357.860,357.95,357.8000,357.8200,-0.0589,309294,-0.0589,-0.016458,-183275.0,-3.720799e+01,356.46170,356.487538,356.490823,356.532945,356.664624,357.170727,0.1500,0.0
147038,11/13/20 15:57,357.825,358.21,357.8118,358.2050,0.3850,479111,0.3850,0.107596,169817.0,5.490472e+01,356.49370,356.502538,356.509285,356.543445,356.675824,357.179803,0.3982,1.0
147039,11/13/20 15:58,358.205,358.35,358.1400,358.3500,0.1450,718921,0.1450,0.040480,239810.0,5.005312e+01,356.52570,356.525087,356.528131,356.556445,356.683026,357.189653,0.2100,1.0


In [ ]:
# Creating new DataFrame that excludes the first row of data
df_final=df.iloc[1:,:]
df_final.rename(columns={"Time":"time","Open":"open","High":"high","Low":"low","Close":"close","Volume":"volume", "open_close_change_$":"intraday_change_$"}, inplace=True)
df_final

In [ ]:
df["change_category"]=

In [103]:
# Save DataFrame as CSV file
df_final.to_csv("intraday_transformed_2.csv", index=False)